# Interesting Points
This notebook reivews the ability to generate interesting points from processed imagery

### Import some libraries, configure Django

In [1]:
import os
import django
import pandas as pd
from glob import glob
from time import time
import geopandas as gpd
import subprocess
from shapely.wkt import loads

import sys; sys.path.append('../../')
os.environ['DJANGO_SETTINGS_MODULE'] = 'gaia.settings'
django.setup()

from asgiref.sync import sync_to_async
from django.core.management import call_command
from django.contrib.gis.geos import GEOSGeometry

from whale.models import ExtractTransformLoad as ETL
from whale.models import PointsOfInterest as POI

### User defined variables

In [2]:
interesting_points_dir = '../../data/geojson/interesting_points_5-2-2025'

poi_columns = ['id', 'vendor_id', 'sample_idx', 'area', 'deviation', 'epsg_code']

### Identify all GeoJSONs

In [3]:
geojsons = glob(interesting_points_dir + '/**/*.geojson', recursive=True)
geojsons = [geojson.replace('\\', '/') for geojson in geojsons]
geojsons[0]

'../../data/geojson/interesting_points_5-2-2025/10300100BB27E800/21MAR21152059-S1BS-506967347050_01_P001_u08mr32619_cog-pt.geojson'

### Review Interesting Points GeoJSON

In [4]:
gdf = gpd.read_file(geojsons[0])
print(f"The shape of your Geodataframe is: {gdf.shape}\n")
gdf.head()

The shape of your Geodataframe is: (331, 4)



,id,area,deviation,geometry
0,27,690.035112,4.660654,POINT (375124.47 4626903.983)
1,31,6410.979038,5.408343,POINT (374975.473 4626926.06)
2,32,126.950080,83.986252,POINT (375662.516 4626853.718)
3,33,5.460218,6.637307,POINT (376355.989 4626436.534)
4,34,952.125600,29.653807,POINT (379458.586 4626288.054)


### Add Interesting Points to SpatiaLite Database

In [5]:
def import_poi(geojson_path):
    """ Synchronous Import Points of Interest function.

        Takes the GeoJSON filepath and converts the file path to Vendor ID
            using the panchromatic image as the basis for this (opposed to
            the multispectral). Queries the ExtractTransformLoad (ETL) table
            in SpatiaLite for the relevant Vendor ID object. Reads the GeoJSON
            file to a GeoDataFrame. Updates or creates the Interesting Points
            records from a combination of the ETL and GeoJSON information.

        Print statements support troubleshooting.
    """
    cid = geojson_path.split('/')[-2]
    
    vid = '_'.join(geojson_path.split('/')[-1:][0].split('.')[0].split('_')[:-2])
    print(f"Adding points for {vid}")
    # obj = ETL.objects.get(vendor_id=vid)

    epsg_code = geojson_path.split('/')[-1:][0].split('.')[0].split('_')[-2].split('mr')[-1]
    gdf = gpd.read_file(geojson_path)

    for index, row in gdf.iterrows():
        # print(f"Processing row: {row['id']}")
        poi, created = POI.objects.update_or_create(
            sample_idx = row['id'],
            defaults={
                # 'catalog_id': obj.id,
                # 'vendor_id': obj.vendor_id,
                'vendor_id': vid,
                # 'entity_id': obj.entity_id,
                'area': row['area'],
                'deviation': row['deviation'],
                'epsg_code': epsg_code,
                'point': row['geometry'].wkt
            }
        )
        # print(f"\t{'Created' if created else 'Updated'} POI with id: {poi.sample_idx}\n")

    # print('Data imported successfully!')

start = time()

async def import_poi_async(file_path):
    await sync_to_async(import_poi, thread_sensitive=True)(file_path)

import asyncio

if asyncio.get_event_loop().is_running():
    for geojson in geojsons:
        await import_poi_async(geojson)
else:
    asyncio.run(import_poi_async(geojson))

end = round(time() - start, 2)
print("It took {} seconds to load {} GeoJSONs".format(end, len(geojsons))) 

Adding points for 21MAR21152059-S1BS-506967347050_01_P001
Adding points for 21MAR21152113-S1BS-507583593010_01_P001
Adding points for 21MAR21152114-S1BS-507583593010_01_P002
Adding points for 21MAR21152115-S1BS-507583593010_01_P003
Adding points for 21MAR21152116-S1BS-507583593010_01_P004
Adding points for 21APR09152109-S1BS-509404630080_01_P001
Adding points for 21APR09152100-S1BS-509404631020_01_P001
Adding points for 21APR09152101-S1BS-509404631020_01_P002
Adding points for 21APR09152102-S1BS-509404631020_01_P003
Adding points for 21APR09152102-S1BS-509404631020_01_P004
Adding points for 21APR14153620-S1BS-509404630100_01_P003
Adding points for 21APR14153621-S1BS-509404630100_01_P004
Adding points for 21MAR10153430-S1BS-509404630040_01_P003
Adding points for 21MAR10153430-S1BS-509404630040_01_P004
Adding points for 21MAR10153431-S1BS-509404630040_01_P005
Adding points for 21MAR03152422-S1BS-509404630050_01_P002
Adding points for 21MAR03152423-S1BS-509404630050_01_P003
Adding points 

### Confirm that the points were added

In [12]:
objs = await sync_to_async(list)(POI.objects.all())
print(f"Number of POI records in database: {len(objs)}\n")

vid = '_'.join(geojsons[0].split('/')[-1:][0].split('.')[0].split('_')[:-2])

geoms = []
attributes = []
for obj in objs:
    if obj.vendor_id == vid:
        attr_dict = {col: getattr(obj, col) for col in poi_columns}
        attributes.append(attr_dict)
    
        geoms.append(GEOSGeometry(obj.point))

gdf = gpd.GeoDataFrame(attributes, geometry = [loads(g.wkt) for g in geoms])
gdf.head()

Number of POI records in database: 73239



,id,vendor_id,sample_idx,area,deviation,epsg_code,geometry
0,12,21MAR21152059-S1BS-506967347050_01_P001,213,2.730109,5.209881,32619,POINT (376327.096 4625765.432)
1,33,21MAR21152059-S1BS-506967347050_01_P001,376,8.190328,7.354961,32619,POINT (378108.218 4625355.991)
2,169,21MAR21152059-S1BS-506967347050_01_P001,1935,2.388846,12.526270,32619,POINT (380827.435 4622843.362)
3,198,21MAR21152059-S1BS-506967347050_01_P001,2145,20.475819,4.225380,32619,POINT (386380.903 4621795.286)
4,213,21MAR21152059-S1BS-506967347050_01_P001,2233,2.047582,8.491290,32619,POINT (392150.074 4621165.167)


### Identify unique Vendor IDs

In [10]:
objs = await sync_to_async(list)(POI.objects.all())
print(f"Number of POI records in database: {len(objs)}\n")

vendor_ids = list(set([obj.vendor_id for obj in objs]))
print(f"Your unique vendor ids are: {vendor_ids}")

Number of POI records in database: 73239

Your unique vendor ids are: ['21MAR10153430-S1BS-509404630040_01_P004', '24MAR14152140-S1BS-508293707010_01_P002', '24MAR14152131-S1BS-508293707010_02_P004', '21APR09152101-S1BS-509404631020_01_P002', '21APR24154044-S1BS-506967344060_01_P005', '21MAR03152407-S1BS-509404630020_01_P002', '21APR24154043-S1BS-506967344060_01_P004', '21MAR03152408-S1BS-506967347080_01_P003', '21MAR03152436-S1BS-509404631010_01_P004', '21APR24154032-S1BS-506967347100_01_P004', '21MAR03152410-S1BS-506967347080_01_P005', '21APR24154014-S1BS-506967348010_01_P001', '21APR14153621-S1BS-509404630100_01_P004', '21APR14153620-S1BS-509404630100_01_P003', '21MAR10153430-S1BS-509404630040_01_P003', '21APR24154042-S1BS-506967344060_01_P003', '24MAR14152141-S1BS-508293707010_01_P003', '21MAR03152424-S1BS-509404630050_01_P004', '24MAR14152130-S1BS-508293707010_02_P003', '21MAR10153431-S1BS-509404630040_01_P005', '21MAR21152114-S1BS-507583593010_01_P002', '21MAR03152423-S1BS-509404

In [11]:
vendor_ids_list = [obj.vendor_id for obj in objs]
vendor_dict = {}
for vendor_id in vendor_ids:
    vendor_dict.update({
        vendor_id: vendor_ids_list.count(vendor_id)
    })

pd.DataFrame.from_dict(vendor_dict, orient='index', columns=['poi'])

,poi
21MAR10153430-S1BS-509404630040_01_P004,117
24MAR14152140-S1BS-508293707010_01_P002,683
24MAR14152131-S1BS-508293707010_02_P004,1167
21APR09152101-S1BS-509404631020_01_P002,87
21APR24154044-S1BS-506967344060_01_P005,509
21MAR03152407-S1BS-509404630020_01_P002,5257
21APR24154043-S1BS-506967344060_01_P004,295
21MAR03152408-S1BS-506967347080_01_P003,8175
21MAR03152436-S1BS-509404631010_01_P004,1103
21APR24154032-S1BS-506967347100_01_P004,556


# End